## Exploring the company match dataset using Autolabel

#### Setup the API Keys for providers that you want to use

In [22]:
import os

# provide your own OpenAI API key here
os.environ['OPENAI_API_KEY'] = 'sk-'

## Start the labeling process!

Labeling with Autolabel is a 3-step process:
* First, we specify a labeling configuration (see `config.json` below)
* Next, we do a dry-run on our dataset using the LLM specified in `config.json` by running `agent.plan`
* Finally, we run the labeling with `agent.run`

### First labeling run

In [23]:
import json

from autolabel import LabelingAgent

In [24]:
# load the config
with open('image_classification.json', 'r') as f:
     config = json.load(f)

Let's review the configuration file below. You'll notice the following useful keys:
* `task_type`: `entity_matching` (since it's an entity matching task)
* `model`: `{'provider': 'openai', 'name': 'gpt-3.5-turbo'}` (use a specific OpenAI model)
* `prompt.task_guidelines`: `'You are provided with descriptions of companies from their websites...` (how we describe the task to the LLM)
* `prompt.labels`: `['not duplicate', 'duplicate']` (the full list of labels to choose from)
* `prompt.few_shot_num`: 3 (how many labeled examples to provide to the LLM)

In [25]:
config

{'task_name': 'ImageClassification',
 'task_type': 'classification',
 'dataset': {'label_column': 'label',
  'delimiter': ',',
  'image_url_column': 'image_url'},
 'model': {'provider': 'openai_vision', 'name': 'gpt-4-vision-preview'},
 'prompt': {'task_guidelines': "Given the description of a painting, predict the style of the paining. You will be first shown multiple descriptions and their styles. For the last input, you'll be shown an image along with the description and your job is to predict the style for this input. Your answer must be from one of the following categories:\n{labels}",
  'labels': ['Impressionism',
   'Color Field Painting',
   'Early Renaissance',
   'Fauvism',
   'Minimalism',
   'Romanticism',
   'Mannerism Late Renaissance',
   'Post Impressionism',
   'Contemporary Realism',
   'Pointillism',
   'Ukiyo e',
   'Abstract Expressionism',
   'Analytical Cubism',
   'Art Nouveau Modern',
   'Expressionism',
   'High Renaissance',
   'Cubism',
   'Naive Art Primiti

In [26]:
# create an agent for labeling
agent = LabelingAgent(config=config)

In [27]:
# dry-run -- this tells us how much this will cost and shows an example prompt
from autolabel import AutolabelDataset
ds = AutolabelDataset("test.csv", config=config)
agent.plan(ds)

Output()

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $5.3897 │
│ Number of Examples       │ 259     │
│ Average cost per example │ $0.0208 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

{"text": "Given the description of a painting, predict the style of the paining. You will be first shown multiple 
descriptions and their styles. For the last input, you'll be shown an image along with the description and your job
is to predict the style for this input. Your answer must be from one of the following 
categories:\nImpressionism\nColor Field Painting\nEarly Renaissance\nFauvism\nMinimalism\nRomanticism\nMannerism 
Late Renaissance\nPost Impressionism\nContemporary Realism\nPointillism\nUkiyo e\nAbstract 
Expressionism\nAnalytical Cubism\nArt Nouveau Modern\nExpressionism\nHigh Renaissance\nCubism\nNaive Art 
Primitivism\nRococo\nPop Art\nSynthetic Cubism\nRealism\nSymbolism\nNorthern Renaissance\nBaroque\nNew 
Realism\n\nYou will return the answer with just one element: \"the correct label\"\n\nSome examples with their 
output answers are provided below:\n\nInput: Description: maurice utrillo la butte pinson 1 \nOutput: Post 
Impressionism\nInput: Description: vincent van gogh le moulin de la galette 1886 1 \nOutput: Post 
Impressionism\nInput: Description: maurice utrillo vase with flowers 1 \nOutput: Post Impressionism\nInput: 
Description: maurice de vlaminck suresnes \nOutput: Cubism\nInput: Description: maurice esteve broucl 1973 1 
\nOutput: Color Field Painting\n\nNow I want you to label the following example:\nInput: Description: maurice 
utrillo moulin de la galette 3 \nOutput: ", "image_url": 
"https://autolabel-benchmarking.s3.amazonaws.com/painting-style-classification/maurice-utrillo_moulin-de-la-galette
-3_jpg.rf.7572fbf1699f459fc883c19e01caf82a.jpg"}

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [28]:
# now, do the actual labeling
ds = agent.run(ds, max_items=10)

Output()

/Users/dhruva/refuel-ai/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dhruva/refuel-ai/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/dhruva/refuel-ai/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


classification_report:
                    precision    recall  f1-score   support

Art Nouveau Modern       0.00      0.00      0.00         1
     Expressionism       0.33      1.00      0.50         1
     Impressionism       0.67      1.00      0.80         2
Post Impressionism       1.00      0.50      0.67         2
           Realism       1.00      0.50      0.67         2
            Rococo       1.00      1.00      1.00         1
       Romanticism       1.00      1.00      1.00         1

          accuracy                           0.70        10
         macro avg       0.71      0.71      0.66        10
      weighted avg       0.77      0.70      0.68        10

Actual Cost: 0.0

┏━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ accuracy ┃ support ┃ completion_rate ┃
┡━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ 0.7      │ 10      │ 1.0             │
└──────────┴─────────┴─────────────────┘